In [1]:
import os
import cv2
from facenet_pytorch import MTCNN
from PIL import Image
import numpy as np
import torch
from torchvision import transforms
import torch.nn as nn
from torchvision import models
import csv

# 定义情绪分类模型
class EmotionClassifier(nn.Module):
    def __init__(self, num_classes=7):  # 假设有7种情绪
        super(EmotionClassifier, self).__init__()
        self.densenet = models.densenet121(pretrained=True)
        num_ftrs = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.densenet(x)

# 加载模型
model = EmotionClassifier().to('cuda')
model.load_state_dict(torch.load('best_model.pt'))
model.eval()

# 定义图片的预处理
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 预测情绪的函数
def predict_emotion(model, face_img):
    image_tensor = transform(face_img).unsqueeze(0).to('cuda')
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        return probabilities.cpu().numpy()

# 人脸检测和裁剪
def detect_and_crop_face(image_path):
    mtcnn = MTCNN(keep_all=True, device='cuda')
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    boxes, _ = mtcnn.detect(img)
    if boxes is not None and len(boxes) > 0:
        areas = [(box[2]-box[0])*(box[3]-box[1]) for box in boxes]
        max_area_idx = np.argmax(areas)
        box = boxes[max_area_idx].astype(int)
        if (box[2] - box[0]) > 0 and (box[3] - box[1]) > 0:
            cropped_face = img[box[1]:box[3], box[0]:box[2]]
            return [cropped_face]
    return None

def process_images_in_folder(folder_path):
    supported_extensions = ['.png', '.jpg', '.jpeg']
    emotion_labels = ['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']
    # 准备CSV文件
    headers = ['image_id'] + [f'image_{emotion}' for emotion in emotion_labels]
    with open('predictions.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)
        
        for root, dirs, files in os.walk(folder_path):
            for filename in files:
                if any(filename.lower().endswith(ext) for ext in supported_extensions):
                    # 使用子文件夹名和文件名构建photo_id
                    subfolder_name = os.path.basename(root)
                    photo_id = f"{subfolder_name}_{filename}"
                    image_path = os.path.join(root, filename)
                    print(f"Processing {image_path}...")
                    cropped_faces = detect_and_crop_face(image_path)
                    if cropped_faces is not None:
                        for face in cropped_faces:
                            if face.size > 0:
                                face_img = Image.fromarray(face)
                                if face_img.width > 0 and face_img.height > 0:
                                    probabilities = predict_emotion(model, face_img)
                                    row = [photo_id] + probabilities.flatten().tolist()
                                    writer.writerow(row)
                                    print(f"Processed and written to CSV: {photo_id}")
                                else:
                                    print("Detected face has invalid dimensions.")
                    else:
                        print("No faces were detected.")

# 指定你的图片文件夹路径并执行处理
folder_path = 'Image'  # 替换为你的图片文件夹路径
process_images_in_folder(folder_path)


c:\Users\vopo\anaconda3\envs\project1\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vopo\anaconda3\envs\project1\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Face 1: neutral with probability 0.8946856260299683
